In [2]:
from loadTDMS import *
# from fft_test import *
from fft_test_split1overlap1 import *

import numpy as np
import pandas as pd
from dask import dataframe as dd
import matplotlib.pyplot as plt
import itertools
import os

## ファイル名変更 ex) AE_18.tdms -> AE_1.tdms
### プログラム書きやすいように

In [35]:
# os.rename(変更前ファイル、変更後ファイル)

for i in range(1261,1533,2):
    num = int((i-15)/2) - 22
    old_name = rf"Z:\100 実験データ（RAW）\20220630---切削実験---AE+切削抵抗\AE\1-5000\AE_{i}.tdms"
    new_name = rf"Z:\100 実験データ（RAW）\20220630---切削実験---AE+切削抵抗\AE\1-5000\OnlyCuttingData\AE_{num}.tdms"
    os.rename(old_name, new_name)
#     print(i,num)

# ①生データ から2^14個取ってくる
## 中間データ①

In [41]:
# AE_{num}.tdms -> AE_extract_{num}.csv
# 生データ から2^14個取ってくる

def extract_16384(num):
    load_path = rf"Z:\100 実験データ（RAW）\20220630---切削実験---AE+切削抵抗\AE\1-5000\OnlyCuttingData\AE_{num}.tdms"
    save_path = rf"Z:\200 実験データ加工\20220630\TrainingData\extract\AE_extract_{num}.csv"
    d = load_tdms(load_path, ch_num=3)
    d = d[0][5000000:5000000+16384]
    pd.DataFrame(d).to_csv(save_path)

In [45]:
for i in range(1,737):
    extract_16384(i)

# ②正規化する前のFFT
## 中間データ②作成

In [3]:
def FFT_withoutNorm(num):
    
    #データ読み込み
#     d = pd.read_csv(rf'Z:\200 Produced_data\20220630\TrainingData\extract\AE_extract_{num}.csv')
#     d = d['0']
    
    
    #inoue_source
    d = pd.read_csv(rf"Z:\inoue_source\2020\A_30m_破断なし\AE信号\AE_{num}.csv")
    d = d[100000:100000+16384]
    d = d.to_numpy()
    d = d.reshape(-1)
    
#     print(d.shape)
    
    #FFT条件
    dt = 1/10**6
    split_t_r = 1 # 1つの枠で全体のどの割合のデータを分析するか。
    overlap = 1  # オーバーラップ率
    window_F = "hamming"  # 窓関数選択: hanning, hamming, blackman
    y_label = "amplitude"
    y_unit = "V"
    N = len(d)
    fq = np.linspace(0, N*dt, N)  # 周波数軸　linspace(開始,終了,分割数)
    

    #FFT実行
#     fq_, amp_ = FFT_main(fq, d, dt, split_t_r, overlap, window_F)
    fq_, amp_ = FFT(fq,d,dt,window_F)

    #fig保存ー正規化前
    output_FN = fr"Z:\200 Produced_data\inoue2020_A\FFT\split1,overlap1\fig\FFT_{num}"
    plot_FFT(fq_, amp_, output_FN, y_label, y_unit,num)

    #dBに変換してcsv保存ー正規化前
    amp_db = pd.DataFrame({'db': 20*np.log10(amp_)})
    output_csv = fr"Z:\200 Produced_data\inoue2020_A\FFT\split1,overlap1\csv\FFT_{num}.csv"
    amp_db.to_csv(output_csv, header=False, index=False)


In [4]:
%%time
for i in range(334,336):
    print(i)
    FFT_withoutNorm(i)

334
335
Wall time: 10.2 s


# FFT(正規化）
## 学習（trarin）データ作成

In [49]:
#学習データから最大最小を算出
#今回は2022年6月の200path-300path
def get_minmax(start,end):
    all_data = []
    for i in range(start,end+1):
        d = pd.read_csv(rf"Z:\200 実験データ加工\20220630\TrainingData\FFT\csv\FFT_{i}.csv")
        d = d.to_numpy()
        d = np.ravel(d)
        all_data.append(d[:])
    all_data = np.array(all_data)
    all_data = all_data.ravel()
    min_d = all_data.min()
    max_d = all_data.max()
    return min_d, max_d

#正規化_minmax
def normalize_minmax(min_, max_, d):
    min_max_d = (d - min_) / (max_ - min_)
    return min_max_d    

In [50]:
min_, max_ = get_minmax(200,300)

In [55]:
%%time
for i in range(736,737):
    d = np.loadtxt(rf"Z:\200 実験データ加工\20220630\TrainingData\FFT\csv\FFT_{i}.csv", delimiter=',')  # 区切り文字としてカンマを指定
    d_norm = normalize_minmax(min_,max_,d)
    amp_db_norm = pd.DataFrame({'db': d_norm})
    output_csv = rf"Z:\200 実験データ加工\20220630\TrainingData\FFT_Norm\csv\FFT_Norm_{i}.csv"
    amp_db_norm.to_csv(output_csv, header=False, index=False)

Wall time: 62 ms


# -------------------------------------------------------------------------------------
# --------------------------------------------------------------------------------------
# 2021年11月

# 検証データ(test data)作成

## ファイル名変更 ex) AE_18.tdms -> AE_1.tdms
### プログラム書きやすいように

In [59]:
for i in range(24,506,2):
    num = int((i-22)/2)
    old_name = rf"Z:\100 実験データ（RAW）\20211115---切削実験---AE+切削抵抗\AE\1-5000\AE_{i}.tdms"
    new_name = rf"Z:\100 実験データ（RAW）\20211115---切削実験---AE+切削抵抗\AE\1-5000\OnlyCuttingData\AE_{num}.tdms"
    os.rename(old_name, new_name)
#     print(i,num)

# ①生データ から2^14個取ってくる
## 中間データ①

In [63]:
# AE_{num}.tdms -> AE_extract_{num}.csv
# 生データ から2^14個取ってくる

def extract_16384(num):
    load_path = rf"Z:\100 実験データ（RAW）\20211115---切削実験---AE+切削抵抗\AE\1-5000\OnlyCuttingData\AE_{num}.tdms"
    save_path = rf"Z:\200 実験データ加工\20211115---切削実験---AE+切削抵抗\TrainingData\extract\AE_extract_{num}.csv"
    d = load_tdms(load_path, ch_num=3)
    d = d[0][5000000:5000000+16384]
    pd.DataFrame(d).to_csv(save_path)

In [65]:
%%time
for i in range(1,242):
    extract_16384(i)

Wall time: 9min 41s


# ②正規化する前のFFT
## 中間データ②作成

In [66]:
def FFT_withoutNorm(num):
    
    #データ読み込み
    d = pd.read_csv(rf'Z:\200 実験データ加工\20211115---切削実験---AE+切削抵抗\TrainingData\extract\AE_extract_{num}.csv')
    d = d['0']
    
    #FFT条件
    dt = 1/10**6
    split_t_r = 0.1 # 1つの枠で全体のどの割合のデータを分析するか。
    overlap = 0.5  # オーバーラップ率
    window_F = "hamming"  # 窓関数選択: hanning, hamming, blackman
    y_label = "amplitude"
    y_unit = "V"
    N = len(d)
    fq = np.linspace(0, N*dt, N)  # 周波数軸　linspace(開始,終了,分割数)
    

    #FFT実行
    fq_, amp_ = FFT_main(fq, d, dt, split_t_r, overlap, window_F)

    #fig保存ー正規化前
    output_FN = rf"Z:\200 実験データ加工\20211115---切削実験---AE+切削抵抗\TrainingData\FFT\fig\FFT_{num}"
    plot_FFT(fq_, amp_, output_FN, y_label, y_unit,num)

    #dBに変換してcsv保存ー正規化前
    amp_db = pd.DataFrame({'db': 20*np.log10(amp_)})
    output_csv = rf"Z:\200 実験データ加工\20211115---切削実験---AE+切削抵抗\TrainingData\FFT\csv\FFT_{num}.csv"
    amp_db.to_csv(output_csv, header=False, index=False)

In [67]:
%%time
for i in range(1,242):
    FFT_withoutNorm(i)

Wall time: 2min 6s


# FFT(正規化）
## 検証（test）データ作成

In [68]:
#学習データから最大最小を算出
#今回は2022年6月の200path-300path
def get_minmax(start,end):
    all_data = []
    for i in range(start,end+1):
        d = pd.read_csv(rf"Z:\200 実験データ加工\20220630\TrainingData\FFT\csv\FFT_{i}.csv")
        d = d.to_numpy()
        d = np.ravel(d)
        all_data.append(d[:])
    all_data = np.array(all_data)
    all_data = all_data.ravel()
    min_d = all_data.min()
    max_d = all_data.max()
    return min_d, max_d

#正規化_minmax
def normalize_minmax(min_, max_, d):
    min_max_d = (d - min_) / (max_ - min_)
    return min_max_d    

In [69]:
min_, max_ = get_minmax(200,300)

In [70]:
%%time
for i in range(1,242):
    d = np.loadtxt(rf"Z:\200 実験データ加工\20211115---切削実験---AE+切削抵抗\TrainingData\FFT\csv\FFT_{i}.csv", delimiter=',')  # 区切り文字としてカンマを指定
    d_norm = normalize_minmax(min_,max_,d)
    amp_db_norm = pd.DataFrame({'db': d_norm})
    output_csv = rf"Z:\200 実験データ加工\20211115---切削実験---AE+切削抵抗\TrainingData\FFT_Norm\csv\FFT_Norm_{i}.csv"
    amp_db_norm.to_csv(output_csv, header=False, index=False)

Wall time: 6.42 s


,-2.372684329748153687e-02
0,-0.021137
1,-0.018223
2,-0.015309
3,-0.013042
4,-0.011099
...,...
5049630,0.011241
5049631,0.009299
5049632,0.008975
5049633,0.008975


In [2]:
import seaborn

In [4]:
seaborn.load_dataset(name='mpg')

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger
